In [4]:
from plumber.streams import IBLEidStream
from plumber.pipes import DaartInf, IBLMarkerExtractor, SimpleEnsembler, OverviewPlotter

from plumber import Stream, Pipe, PipeLine
from tqdm import tqdm

artifact_path = "./artifacts"

In [2]:
# Closest setting
# - if CAMERA = "left" then camera is to the left of the mouse (you are in mouse frame of reference)
# -- then the closest paw is the paw="paw_r" as it is in the right hand side of the screen

# marker_type = 'LP' or 'DLC'
# So strong combos:
# view = 'left'
# paw = 'paw_r'

# view = 'right'
# paw = 'paw_r'


# Weak combos:
# view = 'left'
# paw = 'paw_l'

# view = 'right'
# paw = 'paw_l'

In [ ]:
def make_pipeline(idx=1, marker_type = 'DLC', view="left", paw="paw_r"):

    # One stream
    input_stream = IBLEidStream('eid_stream', idx)

    # pipes
    extractor = IBLMarkerExtractor('markers', artifact_path, smooth=False, view=view, paw=paw, mtype=marker_type)
    predictor_1 = DaartInf('Mod1', artifact_path, "mod1")
    predictor_2 = DaartInf('Mod2', artifact_path, "mod2")
    predictor_3 = DaartInf('Mod3', artifact_path, "mod3")
    predictor_4 = DaartInf('Mod4', artifact_path, "mod4")
    predictor_5 = DaartInf('Mod5', artifact_path, "mod5")
    ensembler = SimpleEnsembler('ens')
    plotter = OverviewPlotter('overview', artifact_path, mtype=marker_type, eid=input_stream.data["eid_stream"]["eid"], view=view, paw=paw)

    # 1 pipeline
    pl = PipeLine()
    pl.register(input_stream, extractor)

    pl.register(extractor, predictor_1)
    pl.register(extractor, predictor_2)
    pl.register(extractor, predictor_3)
    pl.register(extractor, predictor_4)
    pl.register(extractor, predictor_5)

    pl.register(predictor_1, ensembler)
    pl.register(predictor_2, ensembler)
    pl.register(predictor_3, ensembler)
    pl.register(predictor_4, ensembler)
    pl.register(predictor_5, ensembler)

    pl.register(ensembler, plotter)

    pl.flow(input_stream)

    return input_stream
    

In [ ]:
# There are 53 eids in the list
for i in tqdm(range(53)): input_stream = make_pipeline(i, view="left", paw="paw_r", marker_type="DLC")